In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(path)


In [3]:
# Remove commas 
df['price'] = df['price'].str.replace(',', '').astype(float)
df['number_of_ratings'] = df['number_of_ratings'].str.replace(',', '').astype(float)
df['price_before_discount'] = df['price_before_discount'].str.replace(',', '')

In [4]:
df['price'].replace(0, np.nan, inplace=True)


In [5]:
df['price'] = df['price'].astype(float)
df['number_of_ratings'] = df['number_of_ratings'].astype(float)


In [6]:
# Count non-null values for each row
df['non_null_count'] = df.notnull().sum(axis=1)

# Sort the DataFrame by 'product_name' and 'non_null_count' in descending order
df_sorted = df.sort_values(by=['product_name', 'non_null_count'], ascending=[True, False])

# Drop duplicates, keeping the first occurrence (which has the most non-null values)
df_cleaned = df_sorted.drop_duplicates(subset='product_name', keep='first')

# Drop the temporary 'non_null_count' column
df_cleaned = df_cleaned.drop(columns='non_null_count')

df=df_cleaned

In [7]:
# Drop rows that have the same name
#df = df.drop_duplicates(subset='product_name', keep='first')

In [8]:
df['product_link'] = df['product_link'].apply(lambda x: f'https://www.amazon.com{x}')
df['price_before_discount'] = df['price_before_discount'].str.replace('$', '').astype(float)
df['price'] = df['price'].astype(float)
# Calculating discount where price_before_discount is non-zero and non-null
df['discount_percentage'] = ((df['price_before_discount'] - df['price']) / df['price_before_discount']) * 100

# Replace discount with 0 where price_before_discount is missing or zero to avoid errors
df['discount_percentage'] = df['discount_percentage'].fillna(0).replace([float('inf'), -float('inf')], 0)


C:\Users\mikel\AppData\Local\Temp\ipykernel_18948\3495260639.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['price_before_discount'] = df['price_before_discount'].str.replace('$', '').astype(float)


In [9]:
# Drop rows where all of the specified columns have missing values
df = df.dropna(subset=['RAM', 'storage', 'operating_system','cellular_technology','CPU'], how='all')


In [10]:
# Calculate the columns that are not in the first part of the columns_order list
extra_columns = [col for col in df.columns if col not in [
    'product_name',
    'product_link',
    'image_link',
    'price',
    'discount_percentage',
    'price_before_discount',
    'rating',
    'number_of_ratings'
]]

# Define the full columns order
columns_order = [
    'product_name',
    'product_link',
    'image_link',
    'price',
    'discount_percentage',
    'price_before_discount',
    'rating',
    'number_of_ratings'
] + extra_columns

# Reordering the DataFrame columns
df = df[columns_order]

In [11]:
# Rename columns
df = df.rename(columns={
    'rating': 'rating_out_of_5',
    'RAM': 'RAM (GB)',
    'storage': 'Storage (GB)',
    'screen_size': 'screen_size (Inches)',
})

In [12]:
df['RAM (GB)'] = df['RAM (GB)'].replace('4 tablespoon\(s\)', np.nan, regex=True)
df['RAM (GB)'] = df['RAM (GB)'].replace('6 32nds', np.nan)
df['RAM (GB)'] = df['RAM (GB)'].replace('6 Accelerated Reader', np.nan)
df['Storage (GB)'] = df['Storage (GB)'].replace('300 Milliliters', np.nan)

df['screen_size (Inches)'] = df['screen_size (Inches)'].replace(r'300 Hundredths', np.nan, regex=True)


In [13]:
# Function to convert to int if possible, otherwise to float
def convert_to_int_or_float(value):
    try:
        return int(value)
    except ValueError:
        return float(value)

    
# Define a function to convert 'Gigabits' to GB
def convert_gb_to_gb(value):
    if isinstance(value, str) and 'Gigabits Per Second' in value:
        return float(value.replace('Gigabits Per Second', '').strip()) / 8
    return value    

# Define a function to convert 'MB' values to GB
def convert_mb_to_gb(value):
    if isinstance(value, str) and 'MB' in value:
        return float(value.replace('MB', '').strip()) / 1024
    return value

# Define a function to remove "Units"
def delete_unit(value):
    if isinstance(value, str) and 'Units' in value:
        return float(value.replace('Units', '').strip())
    return value

# Define a function to convert 'TB' values to GB
def convert_tb_to_gb(value):
    if isinstance(value, str) and 'TB' in value:
        return float(value.replace('TB', '').strip()) * 1024
    return value

# Define a function to convert 'Centimeters' to Inches
def convert_cm_to_inches(value):
    if isinstance(value, str) and 'Centimeters' in value:
        return float(value.replace('Centimeters', '').strip()) * 0.393701
    return value

def convert_feet_to_inches(value):
    if isinstance(value, str) and 'Feet' in value:
        return float(value.replace('Feet', '').strip()) * 12
    return value

def convert_mm_to_inches(value):
    if isinstance(value, str) and 'Millimeters' in value:
        return float(value.replace('Millimeters', '').strip()) * 0.0393701
    return value

def convert_meters_to_inches(value):
    if isinstance(value, str) and 'Meters' in value:
        return float(value.replace('Meters', '').strip()) * 39.3701
    return value



df['RAM (GB)'] = df['RAM (GB)'].str.replace('A/W', '').str.strip()
df['Storage (GB)'] = df['Storage (GB)'].str.replace('A/W', '').str.strip()

df['RAM (GB)'] = df['RAM (GB)'].str.replace('Gigabytes Per Second', 'GB').str.strip()
df['Storage (GB)'] = df['Storage (GB)'].str.replace('Gigabytes Per Second', 'GB').str.strip()

# Clean the 'rating_out_of_5' column: Extract the numeric value from strings like "4.1 out of 5 stars"
df['rating_out_of_5'] = df['rating_out_of_5'].str.extract(r'(\d+\.\d+|\d+)').astype(float)

# Apply the conversion function
df['number_of_ratings'] = df['number_of_ratings'].apply(convert_to_int_or_float)

# Clean and convert 'RAM (GB)' column
df['RAM (GB)'] = df['RAM (GB)'].str.replace('GB', '')
# Clean and convert 'Storage (GB)' column
df['Storage (GB)'] = df['Storage (GB)'].str.replace('GB', '')

# Convert MB to GB and TB to GB for RAM and Storage
df['RAM (GB)'] = df['RAM (GB)'].apply(convert_mb_to_gb)
df['RAM (GB)'] = df['RAM (GB)'].apply(convert_tb_to_gb)
df['RAM (GB)'] = df['RAM (GB)'].apply(convert_gb_to_gb)
df['RAM (GB)'] = df['RAM (GB)'].apply(delete_unit)
                                      
df['Storage (GB)'] = df['Storage (GB)'].apply(convert_mb_to_gb)
df['Storage (GB)'] = df['Storage (GB)'].apply(convert_tb_to_gb)

# Apply the conversion function
df['RAM (GB)'] = df['RAM (GB)'].apply(convert_to_int_or_float)
df['Storage (GB)'] = df['Storage (GB)'].apply(convert_to_int_or_float)

# Clean 'screen_size (Inches)' column: Remove 'Inches' and 'cm', convert to float and inches
df['screen_size (Inches)'] = df['screen_size (Inches)'].str.replace('Inches', '')
df['screen_size (Inches)'] = df['screen_size (Inches)'].apply(convert_cm_to_inches)
df['screen_size (Inches)'] = df['screen_size (Inches)'].apply(convert_feet_to_inches)
df['screen_size (Inches)'] = df['screen_size (Inches)'].apply(convert_mm_to_inches)
df['screen_size (Inches)'] = df['screen_size (Inches)'].apply(convert_meters_to_inches)

df['screen_size (Inches)'] = df['screen_size (Inches)'].astype(float)


In [14]:
# Define a function to round discount to 2 decimal places
def round_discount(value):
    try:
        return round(float(value), 2)
    except ValueError:
        return value

# Apply the function to round 'discount' column to 2 decimal places
df['discount_percentage'] = df['discount_percentage'].apply(round_discount)
df['price_before_discount'] = df['price_before_discount'].apply(round_discount)

In [15]:
df.rename(columns={'price': 'Price (Dollar)'}, inplace=True)


In [16]:
df = df.sort_values(by='number_of_ratings', ascending=False)


In [17]:
df.insert(0, 'ID', range(1, len(df) + 1))


In [18]:
df.loc[df['discount_percentage'] < 0, 'discount_percentage'] = 0
df.loc[df['discount_percentage'] == 0, 'price_before_discount'] = np.nan

In [19]:
df.to_csv('Data_After_Cleaning.csv', index=False)
